### **Importing the packages**

In [ ]:
from transformers import AutoTokenizer as at
from transformers import AutoModelForSequenceClassification  as amsc
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### **Initiating the pre-trained [BERT] model**

In [ ]:
tokenizer = at.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')   #Loading the pre-trained NLP model 
model = amsc.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

###**Analysing , encoding and calculating sentiment. [PIPELINE]**

In [ ]:
token = tokenizer.encode("I love your service  ",return_tensors='pt') #An example review is passed "I love your service".
                                                                      #return_tensor='pt' will return tensors instead of list of python integers

In [ ]:
result = model(token)  #passing our tokens to our model 

In [ ]:
result.logits   #This is a vector of raw predictions that a classification model generates.

In [ ]:
ratings = int(torch.argmax(result.logits)+1)   #using pytorch to normalize the vector of raw predictions
print(ratings)

### **Collecting online reviews from websites [WEB SCRAPING] using beautiful soup or you can use freely available datasets**

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')   #I am doing webscraping 
soup = BeautifulSoup(r.text , 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})

In [ ]:
results     #Awesome we got all the reviews but they have some unwanted things with it .we need to clean it .

In [ ]:
reviews = [result.text for result in results]
reviews   #clear view of data(reviews)

### **Loading reviews into the data frame**

In [ ]:
data_frame = pd.DataFrame(np.array(reviews),columns =['Reviews'])    #adding the cleansed data into a dataframe

In [ ]:
data_frame            #see now ! . This looks amazing and structured .

In [ ]:
data_frame['Reviews'].iloc[0] #Just viewing 

### **Creating a function using the pipeline**

In [ ]:
def sentiment_analyzer(Reviews):
  token = tokenizer.encode(Reviews,return_tensors='pt')             #A function to do the work of our pipeline
  result = model(token)            
  return int(torch.argmax(result.logits)+1)  

### **Calling the function  and computing the sentiment score of the customer**

In [ ]:
sentiment_analyzer(data_frame['Reviews'].iloc[0])  #An illustration for single review.   

### **Looping through all the reviews and adding the sentiment scores to the data frame**

In [ ]:
data_frame['Sentiment_Score'] = data_frame['Reviews'].apply(lambda x: sentiment_analyzer(x[:1000])) 

In [ ]:
data_frame #final 

### **This is it . So we have analyzed and now we have the sentiments of the customers in our dataframe .(1-5) Where , 1 is the worst and 5 is the best .As, you can see above . So, by using this you can do whatever you want, like you can  create awesome visualizations or anything that you want to do .**